In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/X.csv


In [2]:
sys.argv[1] = "../input/X.csv"

'''# Test data import
data = np.genfromtxt(sys.argv[1], delimiter = ",")'''

'# Test data import\ndata = np.genfromtxt(sys.argv[1], delimiter = ",")'

In [3]:
import numpy as np
import pandas as pd
import scipy as sp
import sys


def mu_init(data):
    # Randomly selects K observations in data as initial cluster centroids 
    n, d = np.shape(data)
    mu = data[np.random.choice(n, size = K, replace = False)]
    return mu


def KM_eucl_distance_to_centroids(data, mu):
    # Returns the matrix of Euclidean distance of each observation to the K centroids (mu)
    n, d = np.shape(data)
    eucl_distance_to_centroids = np.zeros([n, K])
    idx = 0
    for centroid in mu:
        eucl_distance_to_centroids[:, idx] = np.sqrt(np.sum((data - mu[idx])**2, axis = 1))
        idx += 1
    return eucl_distance_to_centroids


def KM_cluster_assign(eucl_distance_to_centroids):
    # returns vector C of cluster assignment for each observation in data. Cluster is assigned as the shortest Euclidean distance to a cluster centroid.
    C = np.argmin(eucl_distance_to_centroids, axis=1)
    return C


def KM_centroids_update(data, C):
    # Returns the position for the K centroids based on the clustering in vector C
    n, d = np.shape(data)
    mu = np.zeros([K, d])
    for k in range(K):
        data_extract = data[np.where(C == k)[0]]
        mu[k] = np.mean(data_extract, axis = 0)
    return mu


def KM_obj_fun(data, eucl_distance_to_centroids, C):
    # Returns the K-mean objective function value
    n, d = np.shape(data)
    L = 0 # intialization
    for i in range(n):
        L += eucl_distance_to_centroids[i, C[i]]
    return L


def KMeans(data):
    #perform the algorithm with 5 clusters and 10 iterations...you may try others for testing purposes, but submit 5 and 10 respectively
    mu = mu_init(data) # Initialize centroids
    L = [] # Initialize objective function (sum over clusters of the Euclidean distances to centroids)
    for i in range(ITERATIONS):
        dist = KM_eucl_distance_to_centroids(data, mu)
        C = KM_cluster_assign(dist)
        mu = KM_centroids_update(data, C)
        centerslist = mu
        L.append((i, KM_obj_fun(data, dist, C)))
        filename = "centroids-" + str(i+1) + ".csv" #"i" would be each iteration
        np.savetxt(filename, centerslist, delimiter=",")
    return L, C, mu


def EMGMM_phi(data, pi, mu, sigma):
    n, d = np.shape(data)
    phi = np.zeros([n, K])
    for i in range(n):
        for k in range(K):
            phi[i, k] = pi[k] * np.linalg.det(sigma[k])**(-1/2) * np.exp((-1/2) * np.dot(np.dot((data[i] - mu[k]), np.linalg.inv(sigma[k])), np.transpose(data[i] - mu[k])))
    phi = phi / np.sum(phi, axis=1)[:, None]
    return phi


def EMGMM_pi_update(data, phi, n_K):
    n, d = np.shape(data)
    pi = n_K / n
    return pi


def EMGMM_mu_update(data, phi, n_K):
    n, d = np.shape(data)
    mu = np.zeros([K, d])
    for k in range(K):
        phi_k = phi[:, k]
        mu[k] = (1/ n_K[k]) * np.sum(phi_k[:, None] * data, axis=0) 
    return mu


def EMGMM_sigma_update(data, phi, n_K, mu):
    n, d = np.shape(data)
    sigma = []
    for k in range(K):
        sigma_sum = np.zeros([d, d])
        for i in range(n):
            sigma_sum += phi[i, k] * np.dot((data[i]-mu[k]).reshape(d, 1), (data[i]-mu[k]).reshape(1, d))
        sigma.append((1 / n_K[k]) * sigma_sum)
    return sigma

    
def EMGMM(data):
    # Applies Exppactation-Maximization algorithm to learn a Gaussian Mixture Model
    n, d = np.shape(data) # dimensions of the matrix data 
    
    mu = mu_init(data) # initialize mean vectors
    pi = np.ones(5) * (1 / K) # initialize prior distribution
    sigma = []
    for k in range(K): # initialize covariances (sigmak_k) for each cluster
        sigma.append(np.eye(d)) 
    
    for i in range(ITERATIONS):
        phi = EMGMM_phi(data, pi, mu, sigma)
        n_K = np.sum(phi, axis=0)
        pi = EMGMM_pi_update(data, phi, n_K)
        mu = EMGMM_mu_update(data, phi, n_K)
        sigma = EMGMM_sigma_update(data, phi, n_K, mu)
        
        filename = "pi-" + str(i+1) + ".csv"
        np.savetxt(filename, pi, delimiter=",")
        filename = "mu-" + str(i+1) + ".csv"
        np.savetxt(filename, mu, delimiter=",")  #this must be done at every iteration
        for j in range(K): #K is the number of clusters
            filename = "Sigma-" + str(j+1) + "-" + str(i+1) + ".csv" #this must be done 5 times (or the number of clusters) for each iteration
            np.savetxt(filename, sigma[j], delimiter=",")
    return pi, mu, sigma,


K = 5 # number of clusters
ITERATIONS = 10 # number of iterations


if __name__ == '__main__':
    data = np.genfromtxt(sys.argv[1], delimiter = ",")
    KMeans(data)
    EMGMM(data)

In [4]:
# data = data[0:15] #create a small dataset for testing purpose

In [5]:
# Test K-means clustering
L, C, mu = KMeans(data)
print('Objective function for each iteration = \n', L, '\n')
print('Final clustering = \n', C, '\n')
print('Final centroids = \n', mu, '\n')

Objective function for each iteration = 
 [(0, 1065.6523156278733), (1, 880.4921313290795), (2, 871.4848168683961), (3, 867.8528826316434), (4, 866.7398455917674), (5, 866.4627795919249), (6, 866.5782764401727), (7, 866.6841256628805), (8, 866.7937133499829), (9, 866.8531063439523)] 

Final clustering = 
 [2 2 1 1 0 1 1 1 0 3 0 3 2 2 2 0 4 2 0 2 0 3 1 1 4 2 2 4 1 2 2 2 0 3 1 4 4
 1 2 1 3 1 3 3 3 1 1 4 3 2 3 2 0 4 1 1 2 4 2 0 2 1 1 1 4 1 4 1 4 1 1 2 3 2
 4 0 4 2 1 3 4 4 1 4 4 1 4 1 3 4 1 1 2 2 0 1 2 4 2 3 3 1 0 1 1 2 3 1 4 0 3
 4 3 4 2 3 0 3 4 4 0 3 0 4 0 1 1 4 4 1 4 0 0 0 3 3 0 0 3 1 3 1 3 1 3 3 3 2
 3 0 4 3 1 3 3 4 3 1 0 1 4 1 3 4 4 1 2 3 2 4 3 1 1 1 4 3 4 4 3 1 1 0 0 4 2
 3 3 1 0 1 2 2 1 3 3 1 4 2 1 3 2 3 2 1 4 1 3 1 4 4 4 2 3 1 1 0 1 1 0 2 3 4
 2 1 2 1 3 4 3 2 1 2 1 1 3 3 0 3 4 1 2 0 1 1 1 4 1 0 4 3 4 1 2 2 1 3 1 3 4
 0 4 0 3 0 1 0 2 0 3 0 3 4 2 2 0 1 4 3 1 3 4 4 2 0 3 1 0 3 4 3 2 1 1 1 1 4
 3 4 4 3 1 2 2 1 2 1 3 2 1 2 1 1 4 0 3 4 2 3 2 1 3 1 3 1 4 0 2 0 0 4 3 2 1
 3 1 0 2 1 3 4 0 2

In [6]:
# Test Expectation-Maximization of Gaussain Mixture Model
pi, mu, sigma = EMGMM(data)
print('Final pi = \n', pi, '\n')
print('Final mu = \n', mu, '\n')
print('Final sigma= \n', sigma, '\n')

Final pi = 
 [0.00683575 0.56630608 0.18852039 0.15473363 0.08360414] 

Final mu = 
 [[0.71294927 1.46013881 0.34719263 0.47630234 1.49651464 1.68399278
  3.17358776 2.87793949 2.83863643 2.98769753]
 [0.88355303 1.29217022 0.81656485 0.49637454 0.89287401 1.81726171
  3.18       2.84905867 1.56995787 2.79954002]
 [0.96590878 1.349163   2.00899277 0.54434479 1.07509298 2.14465993
  3.19038433 2.80080248 1.54499942 2.57034077]
 [0.82671872 1.30505603 0.37051221 0.38516396 0.81304911 1.45449325
  3.17005899 2.83873186 1.47643983 3.19838214]
 [0.9323005  1.84902855 1.34491366 1.02777543 1.14087547 2.06794884
  3.18673601 2.80719846 1.68937891 2.55903416]] 

Final sigma= 
 [array([[ 8.41281469e-03,  1.79253143e-03,  7.39122779e-03,
         1.06775166e-03, -5.43690041e-03, -6.85518860e-03,
        -1.45767248e-04, -2.49666883e-03,  6.77032015e-03,
         1.00996059e-02],
       [ 1.79253143e-03,  9.29373704e-02,  5.52591777e-02,
         7.32775251e-03, -1.17985300e-01, -3.76324953e-02,
